In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import lognorm
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.integrate import quad

np.random.seed(42)

lambda_true = 0.5
mu_true = 2.0
sigma_true = 0.4
b_true = 0.8
rho = 0.05
N = 1000
censoring_prob = 0.1

def solve_rV(lambda_, b, mu, sigma, rho):
    w_bar = b
    for _ in range(100):
        w_vals = np.linspace(w_bar, lognorm.ppf(0.999, s=sigma, scale=np.exp(mu)), 200)
        f_vals = lognorm.pdf(w_vals, s=sigma, scale=np.exp(mu))
        integrand = (w_vals - w_bar) * f_vals
        expected_gain = np.trapz(integrand, w_vals)
        w_new = b + (lambda_ / rho) * expected_gain
        if np.abs(w_new - w_bar) < 1e-6:
            break
        w_bar = w_new
    return w_bar

rV_true = solve_rV(lambda_true, b_true, mu_true, sigma_true, rho)

data = []
for _ in range(N):
    if np.random.rand() < censoring_prob:
        t = np.random.uniform(0, 20)
        data.append((t, np.nan, 0))  
    else:
        h = lambda_true * (1 - lognorm.cdf(rV_true, s=sigma_true, scale=np.exp(mu_true)))
        t = np.random.exponential(1 / h)
        while True:
            w = lognorm.rvs(s=sigma_true, scale=np.exp(mu_true))
            if w >= rV_true:
                break
        data.append((t, w, 1))  

df = pd.DataFrame(data, columns=['duration', 'wage', 'employed'])
print(df.head())

    duration       wage  employed
0   6.020259   9.574233         1
1   0.339194  13.588338         1
2  17.323523        NaN         0
3   2.462507   6.123968         1
4   0.405224   9.179971         1


In [ ]:
def solve_rV(lambda_, b, mu, sigma, rho):
    w_bar = b
    for _ in range(100):
        w_vals = np.linspace(w_bar, lognorm.ppf(0.999, s=sigma, scale=np.exp(mu)), 200)
        f_vals = lognorm.pdf(w_vals, s=sigma, scale=np.exp(mu))
        integrand = (w_vals - w_bar) * f_vals
        expected_gain = np.trapz(integrand, w_vals)
        w_new = b + (lambda_ / rho) * expected_gain
        if np.abs(w_new - w_bar) < 1e-6:
            break
        w_bar = w_new
    return w_bar

def neg_log_likelihood(params, df, rho):
    lambda_, mu, sigma, b = params
    if lambda_ <= 0 or sigma <= 0 or b <= 0:
        return np.inf
    rV = solve_rV(lambda_, b, mu, sigma, rho)
    F_rV = lognorm.cdf(rV, s=sigma, scale=np.exp(mu))
    f_rV = lognorm.pdf(df['wage'], s=sigma, scale=np.exp(mu))

    hazard = lambda_ * (1 - F_rV)
    ll = np.zeros(len(df))

    uncensored = df['employed'] == 1
    censored = df['employed'] == 0


    ll[uncensored] = (
        np.log(lambda_) +
        np.log(f_rV[uncensored]) -
        hazard * df['duration'][uncensored]
    )

    ll[censored] = - hazard * df['duration'][censored]

    return -np.sum(ll)

In [5]:
initial_guess = [0.3, 1.5, 0.5, 1.0]
bounds = [(1e-4, 2), (0.5, 3), (0.1, 1), (0.1, 2)]

res = minimize(neg_log_likelihood, initial_guess, args=(df, rho), bounds=bounds)
lambda_hat, mu_hat, sigma_hat, b_hat = res.x

print(f"Estimated λ: {lambda_hat:.4f}")
print(f"Estimated μ: {mu_hat:.4f}")
print(f"Estimated σ: {sigma_hat:.4f}")
print(f"Estimated b: {b_hat:.4f}")

Estimated λ: 0.2947
Estimated μ: 2.0046
Estimated σ: 0.4044
Estimated b: 1.2886


In [ ]:
def compute_reservation_wage(lambda_hat, b_hat, mu_hat, sigma_hat, rho, tol=1e-6, max_iter=100):
    rV = b_hat  
    for _ in range(max_iter):
     
        scale = np.exp(mu_hat)
        s = sigma_hat


        integrand = lambda w: (w - rV) * lognorm.pdf(w, s=s, scale=scale)

        upper = lognorm.ppf(0.999, s=s, scale=scale)
        expected_gain, _ = quad(integrand, rV, upper)

        rV_new = b_hat + (lambda_hat / rho) * expected_gain

        if abs(rV_new - rV) < tol:
            return rV_new
        rV = rV_new

    print("Warning: Reservation wage did not converge.")
    return rV

In [7]:
lambda_hat, mu_hat, sigma_hat, b_hat = res.x

rV_hat = compute_reservation_wage(lambda_hat, b_hat, mu_hat, sigma_hat, rho, max_iter=500)
print(f"Estimated reservation wage: {rV_hat:.4f}")

Estimated reservation wage: 1.3565


In [11]:
def experiment_counterfactual(b, lambda_, mu, sigma, rho):
 
    def solve_rV(lambda_, b, mu, sigma, rho, tol=1e-6, max_iter=100):
        rV = b
        for _ in range(max_iter):
            integrand = lambda w: (w - rV) * lognorm.pdf(w, s=sigma, scale=np.exp(mu))
            upper = lognorm.ppf(0.999, s=sigma, scale=np.exp(mu))
            expected_gain, _ = quad(integrand, rV, upper)
            rV_new = b + (lambda_ / rho) * expected_gain
            if abs(rV_new - rV) < tol:
                return rV_new
            rV = rV_new
        print("Warning: rV did not converge.")
        return rV

 
    rV = solve_rV(lambda_, b, mu, sigma, rho)


    F_rV = lognorm.cdf(rV, s=sigma, scale=np.exp(mu))
    hazard = lambda_ * (1 - F_rV)


    expected_duration = 1 / hazard

    return {
        "b": b,
        "lambda": lambda_,
        "rV": rV,
        "hazard": hazard,
        "expected_duration": expected_duration
    }


In [12]:

baseline = experiment_counterfactual(b_hat, lambda_hat, mu_hat, sigma_hat, rho)


b_high = experiment_counterfactual(b_hat * 1.5, lambda_hat, mu_hat, sigma_hat, rho)


lambda_high = experiment_counterfactual(b_hat, lambda_hat * 1.5, mu_hat, sigma_hat, rho)

# Compare
import pandas as pd
experiments_df = pd.DataFrame([baseline, b_high, lambda_high])
experiments_df.index = ["Baseline", "Higher b", "Higher λ"]
print(experiments_df.round(3))

              b  lambda     rV  hazard  expected_duration
Baseline  1.289   0.295  1.356   0.295              3.393
Higher b  1.933   0.295  1.983   0.295              3.395
Higher λ  1.289   0.442  1.549   0.442              2.262
